In [ ]:
from sklearn import svm, metrics
from skimage.feature import local_binary_pattern
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from google.colab import drive
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta de la carpeta con las imágenes
path = '/content/drive/MyDrive/Estudios/Máster IIR/Segundo cuatrimestre/Proyecto VAI-AAI/chest_xray'

# Categorías
categorias = ['NORMAL', 'PNEUMONIA']

# Definir el modelo Xception con los pesos preentrenados y extrayendo características antes de las capas fully-connected
base_model = Xception(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
# Aquí se podría utilizar base_model directamente, o agregar alguna capa global de pooling si es necesario
model = Model(inputs=base_model.input, outputs=base_model.output)

def cargar_datos(ruta, size=(128, 128)):
    datos = []
    etiquetas = []
    for i, cat in enumerate(categorias):
        carpeta = join(ruta, cat)
        archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
        for archivo in archivos:
            imagen_path = join(carpeta, archivo)
            imagen = cv2.imread(imagen_path)
            if imagen is None:
                print(f'No se pudo leer la imagen: {imagen_path}')
                continue

            # Redimensionar imagen y realizar preprocesamiento necesario para Xception
            imagen = cv2.resize(imagen, size)
            imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
            imagen = img_to_array(imagen)
            imagen = np.expand_dims(imagen, axis=0)
            imagen = preprocess_input(imagen)

            # Extraer características con Xception
            caracteristicas = model.predict(imagen)
            caracteristicas = caracteristicas.flatten()  # Aplanar las características para usarlas en el SVM

            datos.append(caracteristicas)
            etiquetas.append(i)

    if len(datos) == 0 or len(etiquetas) == 0:
        raise ValueError("No se pudieron cargar los datos. Asegúrate de que la ruta y las categorías son correctas.")

    return np.array(datos), np.array(etiquetas)

# Cargar datos de entrenamiento, validación y prueba
datos_entrenamiento, etiquetas_entrenamiento = cargar_datos(join(path, 'train'))
datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'))

# Asegurarse de que hay datos para al menos dos clases
if len(np.unique(etiquetas_entrenamiento)) < 2 or len(np.unique(etiquetas_prueba)) < 2:
    raise ValueError("Se debe cargar datos para al menos dos clases en cada conjunto de datos.")

# Crear y entrenar el modelo SVM
modelo = svm.SVC(gamma='scale')
modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)

# Evaluar el modelo
etiquetas_predichas = modelo.predict(datos_prueba)
precision = metrics.accuracy_score(etiquetas_prueba, etiquetas_predichas) * 100
print(f'Precisión del modelo: {precision:.2f}%')